# Toxicity Classification with KerasNLP
## Table of Contents
* [1. Overview](#1.)
* [2. Configuration](#2.)
* [3. Setup](#3.)
* [4. Import datasets](#4.)
* [5. Data Preprocessing](#5.)
    * [5.1 Train Validation Split](#5.1)
    * [5.2 Create TensorFlow Dataset](#5.2)
* [6. Model Development](#6.)
    * [6.1 Building model](#6.1)
    * [6.2 Training model](#6.2)
    * [6.3 Evaluating model](#6.3)
* [7. Submission](#7.)
* [8. References](#8.)

<font color="red" size="3">If you found it helpful, please don't forget to upvote.</font>

<a id="1."></a>
## 1. Overview
In this notebook, I am going to build a Jigsaw Toxicity Classification Model using [DistilBERT](https://keras.io/api/keras_nlp/models/distil_bert/) from [KerasNLP Library](https://keras.io/api/keras_nlp).

DistilBERT is a distiled version of BERT which leverages Knowledge Distillation, it retrains 97% of language understanding capabilities of original BERT, while being 40% smaller and 60% faster.

KerasNLP is a Library based on Keras that makes it easier to implement NLP appplication by writing only a few lines of code. As you can see below.
```python
def get_model(config):
    encoder = keras_nlp.models.DistilBertBackbone.from_preset(
        "distil_bert_base_en_uncased"
    )
    encoder.trainable = False
    preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset("distil_bert_base_en_uncased")
    inputs = keras.Input(shape=(), dtype=tf.string)
    x = preprocessor(inputs)
    x = encoder(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    output = layers.Dense(6, activation="sigmoid")(x)
    model = keras.Model(inputs, output, name="model")
    model.compile(
        "adam", loss="binary_crossentropy", metrics=["categorical_accuracy", keras.metrics.AUC()]
    )
    return model
```

<a id="2."></a>
## 2. Configuration

In [2]:
class Config:
    batch_size = 128
    validation_split = 0.15
    epochs = 10 # Number of Epochs to train
    model_path = "model.tf"
    output_dataset_path = "../input/toxicity-classification-keras-nlp-output"
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    modes = ["training", "inference"]
    mode = modes[0]
config = Config()

<a id="3."></a>
## 3. Setup

Now install KerasNLP Library and import necessary packages.

In [4]:
pip install keras-nlp --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.1/527.1 kB 8.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.4.0 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
kfp 1.8.20 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.86.0 which is incompatible.
kfp 1.8.20 requires PyYAML<6,>=5.3, but you have pyyaml 6.0 which is incompatible.
gcsfs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2023.4.0 which is incompatible.
cudf 23.4.0 requires p

In [6]:
import pandas as pd
import tensorflow as tf
import pathlib
import random
import string
import re
import sys
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
from nltk.tokenize import TweetTokenizer 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy.stats import rankdata
import json
import keras_nlp

<a id="4."></a>
## 4. Import datasets

In [7]:
!unzip ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
!unzip ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
!unzip ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
!unzip ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip

Archive:  ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
  inflating: train.csv               
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
  inflating: test.csv                
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
  inflating: test_labels.csv         
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
  inflating: sample_submission.csv   


In [8]:
train = pd.read_csv("/kaggle/working/train.csv")
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


<a id="5."></a>
## 5. Data Preprocessing

<a id="5.1"></a>
### 5.1 Train Validation Split

In [11]:
X_train, X_val, y_train, y_val = train_test_split(train["comment_text"], train[config.labels], test_size=config.validation_split)

In [12]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((135635,), (135635, 6), (23936,), (23936, 6))

<a id="5.2"></a>
### 5.2 Create TensorFlow Dataset

In [14]:
def make_dataset(X, y, batch_size, mode):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if mode == "train":
       dataset = dataset.shuffle(batch_size * 4) 
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache().prefetch(tf.data.AUTOTUNE).repeat(1)
    return dataset

In [15]:
train_ds = make_dataset(X_train, y_train, batch_size=config.batch_size, mode="train")
valid_ds = make_dataset(X_val, y_val, batch_size=config.batch_size, mode="valid")

Let's take a look at the format of training data.

In [16]:
for batch in train_ds.take(1):
    print(batch)

(<tf.Tensor: shape=(128,), dtype=string, numpy=
array([b"There's history with me and the other editor that goes way beyond Wikipedia, which is why I can't take him at his word.  But nonetheless I went too far and got spanked, which I probably had coming.  Lesson learned  I'll put my money where my mouth is....If I ever do it again, you can give me a lifetime ban...finito, see ya bye.  I'm willing to stick my neck out if you're willing me work with me.  (talk)",
       b"Hey guys! This article is grossly focused on government numbers for the amount of Christians in different nations!!\nFor example, I'm half-Norwegian and half-Cantonese and I've gotta say... there are between 5-10% Christians in China and it's definitely not around 90% in Norway just because there are that many members in the State church. Will the number of Christians in Norway really fall by about 80% if the State Church is disbanded? Most of them wouldn't even care! It doesn't work that way. The number of Christians i

<a id="6."></a>
## 6. Model Development

<a id="6.1"></a>
### 6.1 Building model

In [17]:
def get_model(config):
    encoder = keras_nlp.models.DistilBertBackbone.from_preset(
        "distil_bert_base_en_uncased"
    )
    encoder.trainable = False
    preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset("distil_bert_base_en_uncased")
    inputs = keras.Input(shape=(), dtype=tf.string)
    x = preprocessor(inputs)
    x = encoder(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    output = layers.Dense(6, activation="sigmoid")(x)
    model = keras.Model(inputs, output, name="model")
    model.compile(
        "adam", loss="binary_crossentropy", metrics=["categorical_accuracy", keras.metrics.AUC()]
    )
    return model

In [18]:
model = get_model(config)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 distil_bert_preprocessor (Dist  {'token_ids': (None  0          ['input_1[0][0]']                
 ilBertPreprocessor)            , 512),                                                           
                                 'padding_mask': (N                                               
                                one, 512)}                                                        
                                                                                                  
 distil_bert_backbone (DistilBe  (None, None, 768)   66362880    ['distil_bert_preprocessor[0]

<a id="6.2"></a>
### 6.2 Training model

In [ ]:
if config.mode == config.modes[0]:
    checkpoint = keras.callbacks.ModelCheckpoint(config.model_path, monitor="val_categorical_accuracy", save_best_only=True)
    early_stopping = keras.callbacks.EarlyStopping(patience=10)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(patience=5, min_delta=1e-4, min_lr=1e-6)
    model.fit(train_ds, epochs=config.epochs, validation_data=valid_ds, callbacks=[checkpoint, reduce_lr])

Epoch 1/10
1005/1060 [===========================>..] - ETA: 1:41 - loss: 0.0936 - categorical_accuracy: 0.8006 - auc: 0.9346

<a id="6.3"></a>
### 6.3 Evaluating model

#### Classification Report

In [ ]:
if config.mode == config.modes[0]:
    from sklearn.metrics import classification_report
    y_pred = np.array(model.predict(valid_ds) > 0.5, dtype=int)
    cls_report = classification_report(y_val, y_pred)
    print(cls_report)

<a id="7."></a>
## 7. Submission

In [ ]:
test = pd.read_csv("/kaggle/working/test.csv")
test.head()

In [ ]:
sample_submission = pd.read_csv("/kaggle/working/sample_submission.csv")
sample_submission.head()

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((test["comment_text"])).batch(config.batch_size).cache().prefetch(1)
path = config.model_path
if config.mode == config.modes[1]:
    path = config.output_dataset_path + "/" + path
model.load_weights(path)
score = model.predict(test_ds)

In [ ]:
sample_submission[config.labels] = score
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head()


<a id="8."></a>
## 8. References
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762v5)
- [DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter](https://arxiv.org/abs/1910.01108)
- [DistilBERT documentation](https://keras.io/api/keras_nlp/models/distil_bert/)